In [89]:
pip install tld

Note: you may need to restart the kernel to use updated packages.


# GoPhish feature extraction

### Importing libraries

In [90]:
import re #regex
import string
import logging

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tldextract import extract as tld_extract
from tld import get_tld, is_tld
from tld.exceptions import TldDomainNotFound, TldBadUrl, TldIOError

### Importing csv file

Edited csv file to only show a value for 1 if the url is malicious, defaced or has malware. The URL will be marked as 0 if it is benign

In [91]:
url_data = pd.read_csv('goPhish_url_data.csv')
url_data.head()

,url,type
0,br-icloud.com.br,1
1,mp3raid.com/music/krizz_kaliko.html,0
2,bopsecrets.org/rexroth/cr/1.htm,0
3,http://www.garage-pirenne.be/index.php?option=...,1
4,http://adventure-nicaragua.net/index.php?optio...,1


In [92]:
url_data.info() #Displaying some information about the csv

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 9.9+ MB


### Checking for null values in dataset

In [93]:
url_data.isnull().sum() #isnull() returns boolean value, .sum() returns the sum of the values

url     0
type    0
dtype: int64

### Extracting count of each value

In [94]:
count = url_data['type'].value_counts()
print(count)

type
0    428103
1    223088
Name: count, dtype: int64


### Normalising the data

Removing 'www.' from the url to normalise the data and extract the featues necessary to detect. 

- 'regex=True' uses the regular expression pattern to find the 'www.' in the whole url string, otherwise it would only find the first instance in the string

In [95]:
url_data['url'] = url_data['url'].replace('www.','', regex=True)

In [96]:
url_data.head()

,url,type
0,br-icloud.com.br,1
1,mp3raid.com/music/krizz_kaliko.html,0
2,bopsecrets.org/rexroth/cr/1.htm,0
3,http://garage-pirenne.be/index.php?option=com_...,1
4,http://adventure-nicaragua.net/index.php?optio...,1


In [97]:
url_data["type"].value_counts()

type
0    428103
1    223088
Name: count, dtype: int64

### Extracting url length

In [98]:
def get_url_length(url):
    prefixes = ['http://','https://'] #Defining common prefixes for URLs
    
    for prefix in prefixes: #Loop to exclude prefix in length of URL
        if url.startswith(prefix):
            url = url[len(prefix):]
    
    url = url.replace('www.','')
    
    return len(url)

In [99]:
url_data['url_len'] = url_data['url'].apply(lambda x: get_url_length(str(x))) #Adding column to show URL length

In [100]:
url_data.head()

,url,type,url_len
0,br-icloud.com.br,1,16
1,mp3raid.com/music/krizz_kaliko.html,0,35
2,bopsecrets.org/rexroth/cr/1.htm,0,31
3,http://garage-pirenne.be/index.php?option=com_...,1,77
4,http://adventure-nicaragua.net/index.php?optio...,1,228


### Extracting top level domain

Extracting the top level domain from the url using the tld module

- 'as_object = true' instructs the function to return a parsed url object
- Cells ran with no error but domains are not found. This could be for multiple reasons. Most likely the domain does not exist on the internet anymore

In [101]:
def extract_tld(url):
    try:
        res = get_tld(url, as_object = True, fail_silenty = False, fix_protocol = True)
        pri_domain = res.parsed_url.netloc
    except:
        pri_domain = None
    return pri_domain

In [102]:
url_data['domain'] = url_data['url'].apply(lambda x: extract_tld(x))

In [103]:
url_data.head()

,url,type,url_len,domain
0,br-icloud.com.br,1,16,None
1,mp3raid.com/music/krizz_kaliko.html,0,35,None
2,bopsecrets.org/rexroth/cr/1.htm,0,31,None
3,http://garage-pirenne.be/index.php?option=com_...,1,77,None
4,http://adventure-nicaragua.net/index.php?optio...,1,228,None


### Extracting character type and count

This involves a few functions that extract 3 different values of the url.

- Count of letters
- Count of digits
- Count of special characters
- Lastly, there will be a list of each of the special characters to identify which ones have been used and if they are suspicious

In [104]:
def count_letters(url):
    num_letters = sum(char.isalpha() for char in url)
    return num_letters

def count_digits(url):
    num_digits = sum(char.isdigit() for char in url)
    return num_digits

def count_special_chars(url):
    special_chars = set(string.punctuation)
    num_special_chars = sum(char in special_chars for char in url)
    return num_special_chars

In [106]:
url_data['letters_count']        = url_data['url'].apply(lambda x: count_letters(x))
url_data['digits_count']         = url_data['url'].apply(lambda x: count_digits(x))
url_data['special_chars_count']  = url_data['url'].apply(lambda x: count_special_chars(x))

In [105]:
feature = ['@','?','-','=','.','#','%','+','$','!','*',',','//']
for i in feature:
    url_data[i] = url_data['url'].apply(lambda x: x.count(i))

In [107]:
url_data.head()

,url,type,url_len,domain,@,?,-,=,.,#,%,+,$,!,*,",",//,letters_count,digits_count,special_chars_count
0,br-icloud.com.br,1,16,None,0,0,1,0,2,0,0,0,0,0,0,0,0,13,0,3
1,mp3raid.com/music/krizz_kaliko.html,0,35,None,0,0,0,0,2,0,0,0,0,0,0,0,0,29,1,5
2,bopsecrets.org/rexroth/cr/1.htm,0,31,None,0,0,0,0,2,0,0,0,0,0,0,0,0,25,1,5
3,http://garage-pirenne.be/index.php?option=com_...,1,77,None,0,1,1,4,2,0,0,0,0,0,0,0,1,60,7,17
4,http://adventure-nicaragua.net/index.php?optio...,1,228,None,0,1,1,3,2,0,0,0,0,0,0,0,1,199,22,14
